In [1]:
import optuna
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

import lightgbm as lgb

/Users/vitor.mello/vho/project_ml_course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_df = pd.read_csv("../data/dados.csv", index_col="Unnamed: 0")

features = ["feat_8", "feat_17", "feat_50"]

X = raw_df[features]
y = raw_df["class"]

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [3]:
def objective(trial):
    param = {
        "objective": "binary",
        "metric": "None",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 7, 128),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
    }
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    for train_idx, valid_idx in cv.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
        lgbm = lgb.LGBMClassifier(**param)
        lgbm.fit(X_tr, y_tr)
        y_pred = lgbm.predict(X_val)
        score = f1_score(y_val, y_pred)
        scores.append(score)
    return sum(scores) / len(scores)


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("Melhores hiperparâmetros encontrados:")
print(study.best_params)
print(f"Melhor F1 médio (CV): {study.best_value:.4f}")


[I 2025-07-16 18:10:52,008] A new study created in memory with name: no-name-8c9c0a57-d39a-4f06-a333-c36c71936da4
Best trial: 0. Best value: 0.860718:   3%|▎         | 1/30 [00:00<00:22,  1.30it/s]

[I 2025-07-16 18:10:52,786] Trial 0 finished with value: 0.8607175517783169 and parameters: {'learning_rate': 0.029100617882474283, 'num_leaves': 63, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.9424143954230736, 'colsample_bytree': 0.6623529010495697, 'reg_alpha': 0.0003734337633494176, 'reg_lambda': 0.003651007189495876}. Best is trial 0 with value: 0.8607175517783169.


Best trial: 0. Best value: 0.860718:   7%|▋         | 2/30 [00:01<00:14,  1.91it/s]

[I 2025-07-16 18:10:53,137] Trial 1 finished with value: 0.7420502467894015 and parameters: {'learning_rate': 0.014445859244816481, 'num_leaves': 122, 'max_depth': 4, 'min_child_samples': 49, 'subsample': 0.7623499909800612, 'colsample_bytree': 0.883576828635398, 'reg_alpha': 2.4857012524562503, 'reg_lambda': 0.00015563974584479592}. Best is trial 0 with value: 0.8607175517783169.


Best trial: 2. Best value: 0.960352:  10%|█         | 3/30 [00:01<00:11,  2.37it/s]

...

[I 2025-07-16 18:11:13,815] Trial 27 finished with value: 0.9712151380635621 and parameters: {'learning_rate': 0.20078428224368222, 'num_leaves': 100, 'max_depth': 6, 'min_child_samples': 13, 'subsample': 0.8959763825317852, 'colsample_bytree': 0.9324406760802145, 'reg_alpha': 0.006863634265334291, 'reg_lambda': 1.0372625044729226e-05}. Best is trial 21 with value: 0.975861027533447.


Best trial: 21. Best value: 0.975861:  97%|█████████▋| 29/30 [00:22<00:00,  1.21it/s]

[I 2025-07-16 18:11:14,312] Trial 28 finished with value: 0.8765182815933337 and parameters: {'learning_rate': 0.022343457080348066, 'num_leaves': 69, 'max_depth': 5, 'min_child_samples': 44, 'subsample': 0.9575242251340672, 'colsample_bytree': 0.9986466334232661, 'reg_alpha': 0.055279474575946196, 'reg_lambda': 2.344889109723794e-05}. Best is trial 21 with value: 0.975861027533447.


Best trial: 21. Best value: 0.975861: 100%|██████████| 30/30 [00:22<00:00,  1.31it/s]

[I 2025-07-16 18:11:14,965] Trial 29 finished with value: 0.9631978524013641 and parameters: {'learning_rate': 0.060057043321136795, 'num_leaves': 62, 'max_depth': 7, 'min_child_samples': 59, 'subsample': 0.9489036154174751, 'colsample_bytree': 0.860197151890462, 'reg_alpha': 0.0026723746248600477, 'reg_lambda': 0.0013753855344367197}. Best is trial 21 with value: 0.975861027533447.
Melhores hiperparâmetros encontrados:
{'learning_rate': 0.2026423734492729, 'num_leaves': 83, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.9419098599507807, 'colsample_bytree': 0.9985816372861133, 'reg_alpha': 2.8281359110848526e-05, 'reg_lambda': 2.243813364676738e-06}
Melhor F1 médio (CV): 0.9759
